# Graph Transformer Model for Oral Bioavailability Dataset 

1. In this notebook, we will build a model based on the graph transformer convolution technique that is available on Pytorch Geometric. 
2. Hyperparameters will be obtained using Optuna library following the Tree-structured Parzen Estimator Algorithm in 30 trials.
3. Models will be trained/validated/tested using 5-fold CV and results will be averaged. 


### Note
1. Ensure that this notebook is in the same working directory as data folder, config.py, utils.py, engine.py and model.py
2. To load models, please download the saved models provided in google drive link from README.md
3. Comment away the training function to load saved models and reproduce results

In [1]:
from utils import seed_everything, LoadHOBDataset
from config import SEED_NO, NUM_FEATURES, NUM_GRAPHS_PER_BATCH, NUM_TARGET, EDGE_DIM, DEVICE, PATIENCE, EPOCHS, N_SPLITS, params_gt
from engine import EngineHOB
from model import GraphTrans

import torch
import numpy as np
import optuna
from sklearn.model_selection import KFold
from torch_geometric.loader import DataLoader
import os 

## 2. Tuning of model (Suggest to skip this step and used the already found hyperparameters saved in config.py)

1. To tune model, we make use of Optuna library to help us find the best parameters in 30 trials using the Tree-structured Parzen Estimator Algorithm.
2. First, create a run_tuning function to include the train and validation step with early stopping mechanism to find the best parameters for each trial
3. Then, create an objective function for Optuna to find the best parameters.

In [5]:
def run_tuning(train_loader, valid_loader, params):
    model = GraphTrans(num_features=NUM_FEATURES, num_targets=NUM_TARGET, num_layers=params['num_layers'], hidden_size=params['hidden_size'], n_heads=params['n_heads'], dropout=params['dropout'], edge_dim=EDGE_DIM)
    model.to(DEVICE)
    optimizer=torch.optim.Adam(model.parameters(),lr = params['learning_rate'])
    eng = EngineHOB(model, optimizer, device=DEVICE)

    best_loss = np.inf
    early_stopping_iter = PATIENCE
    early_stopping_counter = 0 

    for epoch in range(EPOCHS):
        train_loss = eng.train(train_loader)
        valid_loss = eng.validate(valid_loader)
        print(f'Epoch: {epoch+1}/{EPOCHS}, train loss : {train_loss}, validation loss : {valid_loss}')
        if valid_loss < best_loss:
            best_loss = valid_loss 
            early_stopping_counter=0
        else:
            early_stopping_counter +=1

        if early_stopping_counter > early_stopping_iter:
            print('Early stopping...')
            break
        print(f'Early stop counter: {early_stopping_counter}')
    
    return best_loss



In [6]:
def objective(trial):
    params = {
        'num_layers' : trial.suggest_int('num_layers', 1,3),
        'hidden_size' : trial.suggest_int('hidden_size', 64, 512),
        'n_heads' : trial.suggest_int('n_heads', 1, 5),
        'dropout': trial.suggest_float('dropout', 0.1,0.4),
        'learning_rate' : trial.suggest_float('learning_rate', 1e-3, 9e-3, log=True)
    }
    
    #load dataset 
    dataset_for_cv = LoadHOBDataset(root='./data/graph_data/data_oral_avail_train/', raw_filename='data_oral_avail_train_50.csv')
    kf = KFold(n_splits=N_SPLITS)
    fold_loss = 0

    for fold_no, (train_idx, valid_idx) in enumerate(kf.split(dataset_for_cv)):
        print(f'Fold {fold_no}')
        train_dataset= []
        valid_dataset = []
        for t_idx in train_idx:
            train_dataset.append(torch.load(f'./data/graph_data/data_oral_avail_train/processed/molecule_{t_idx}.pt'))
        for v_idx in valid_idx:
            valid_dataset.append(torch.load(f'./data/graph_data/data_oral_avail_train/processed/molecule_{v_idx}.pt'))
        seed_everything(SEED_NO)
        train_loader = DataLoader(train_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True)
        valid_loader = DataLoader(valid_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False)

        loss = run_tuning(train_loader, valid_loader, params)
        fold_loss += loss

    return fold_loss/5


study = optuna.create_study(direction = 'minimize')
study.optimize(objective, n_trials=30)
print(f'best trial:')
trial_ = study.best_trial
print(trial_.values)
print(f'Best parameters: {trial_.params}')

[I 2023-03-15 13:30:33,387] A new study created in memory with name: no-name-bab4f733-f846-4c1e-9b5d-aab8ca74f92d


Fold 0
Epoch: 1/300, train loss : 0.6911721676588058, validation loss : 0.6812695264816284
Early stop counter: 0
Epoch: 2/300, train loss : 0.6876598298549652, validation loss : 0.6729214191436768
Early stop counter: 0
Epoch: 3/300, train loss : 0.6792905032634735, validation loss : 0.682636022567749
Early stop counter: 1
Epoch: 4/300, train loss : 0.6789697259664536, validation loss : 0.6731582283973694
Early stop counter: 2
Epoch: 5/300, train loss : 0.674514576792717, validation loss : 0.6631226539611816
Early stop counter: 0
Epoch: 6/300, train loss : 0.6718787103891373, validation loss : 0.6692750453948975
Early stop counter: 1
Epoch: 7/300, train loss : 0.6740042120218277, validation loss : 0.6720089912414551
Early stop counter: 2
Epoch: 8/300, train loss : 0.6648592799901962, validation loss : 0.6541059017181396
Early stop counter: 0
Epoch: 9/300, train loss : 0.6653721779584885, validation loss : 0.6494285464286804
Early stop counter: 0
Epoch: 10/300, train loss : 0.66064697504

[I 2023-03-15 13:30:50,265] Trial 0 finished with value: 0.64981027841568 and parameters: {'num_layers': 1, 'hidden_size': 161, 'n_heads': 5, 'dropout': 0.3871112302613642, 'learning_rate': 0.0011787497218360226}. Best is trial 0 with value: 0.64981027841568.


Epoch: 19/300, train loss : 0.6503708958625793, validation loss : 0.6889975666999817
Early stopping...
Fold 0
Epoch: 1/300, train loss : 0.6888761818408966, validation loss : 0.6358839869499207
Early stop counter: 0
Epoch: 2/300, train loss : 0.6903033405542374, validation loss : 0.7287794947624207
Early stop counter: 1
Epoch: 3/300, train loss : 0.68180713057518, validation loss : 0.6532896161079407
Early stop counter: 2
Epoch: 4/300, train loss : 0.6880973875522614, validation loss : 0.6693705320358276
Early stop counter: 3
Epoch: 5/300, train loss : 0.6761141568422318, validation loss : 0.7117066383361816
Early stop counter: 4
Epoch: 6/300, train loss : 0.6798339486122131, validation loss : 0.6660518050193787
Early stop counter: 5
Epoch: 7/300, train loss : 0.6744946539402008, validation loss : 0.6573882699012756
Early stop counter: 6
Epoch: 8/300, train loss : 0.6696155220270157, validation loss : 0.680306077003479
Early stop counter: 7
Epoch: 9/300, train loss : 0.6692708730697632

[I 2023-03-15 13:30:59,912] Trial 1 finished with value: 0.6617109656333924 and parameters: {'num_layers': 1, 'hidden_size': 220, 'n_heads': 3, 'dropout': 0.37166155147046764, 'learning_rate': 0.002118823997957276}. Best is trial 0 with value: 0.64981027841568.


...

Epoch: 19/300, train loss : 0.6836668848991394, validation loss : 0.7863814234733582
Early stopping...
Fold 0
Epoch: 1/300, train loss : 1.9879090338945389, validation loss : 0.8197359442710876
Early stop counter: 0
Epoch: 2/300, train loss : 1.2265285402536392, validation loss : 0.7473933696746826
Early stop counter: 0
Epoch: 3/300, train loss : 0.7065126746892929, validation loss : 0.7280492186546326
Early stop counter: 0
Epoch: 4/300, train loss : 0.6934950947761536, validation loss : 0.7240045666694641
Early stop counter: 0
Epoch: 5/300, train loss : 0.6856424063444138, validation loss : 0.6393471956253052
Early stop counter: 0
Epoch: 6/300, train loss : 0.6893102675676346, validation loss : 0.6526715755462646
Early stop counter: 1
Epoch: 7/300, train loss : 0.6750496923923492, validation loss : 0.7918513417243958
Early stop counter: 2
Epoch: 8/300, train loss : 0.6787150502204895, validation loss : 0.6242523193359375
Early stop counter: 0
Epoch: 9/300, train loss : 0.6754160672426

[I 2023-03-15 13:41:06,782] Trial 28 finished with value: 0.6542031526565552 and parameters: {'num_layers': 2, 'hidden_size': 470, 'n_heads': 2, 'dropout': 0.12554599791946428, 'learning_rate': 0.005205407038137495}. Best is trial 7 with value: 0.6388066053390503.


Epoch: 19/300, train loss : 0.6419005542993546, validation loss : 0.723512589931488
Early stopping...
Fold 0
Epoch: 1/300, train loss : 0.7029806971549988, validation loss : 0.6954423785209656
Early stop counter: 0
Epoch: 2/300, train loss : 0.6893594115972519, validation loss : 0.692119836807251
Early stop counter: 0
Epoch: 3/300, train loss : 0.6818856596946716, validation loss : 0.7109052538871765
Early stop counter: 1
Epoch: 4/300, train loss : 0.6780722290277481, validation loss : 0.6662260293960571
Early stop counter: 0
Epoch: 5/300, train loss : 0.6663489043712616, validation loss : 0.6305520534515381
Early stop counter: 0
Epoch: 6/300, train loss : 0.6646373420953751, validation loss : 0.7014908194541931
Early stop counter: 1
Epoch: 7/300, train loss : 0.6531979292631149, validation loss : 0.6502487659454346
Early stop counter: 2
Epoch: 8/300, train loss : 0.6591545641422272, validation loss : 0.6054357886314392
Early stop counter: 0
Epoch: 9/300, train loss : 0.664098829030990

[I 2023-03-15 13:41:19,339] Trial 29 finished with value: 0.6483298778533936 and parameters: {'num_layers': 2, 'hidden_size': 146, 'n_heads': 1, 'dropout': 0.10040528927501444, 'learning_rate': 0.00785459603234657}. Best is trial 7 with value: 0.6388066053390503.


Epoch: 15/300, train loss : 0.6446306854486465, validation loss : 0.7243225574493408
Early stopping...
best trial:
[0.6388066053390503]
Best parameters: {'num_layers': 2, 'hidden_size': 439, 'n_heads': 1, 'dropout': 0.269754753387312, 'learning_rate': 0.007890910361468965}


## 3. Train/validate/test Model 

1. After hyperparameter tuning, the best parameters are saved to config.py as params_gt
2. Next, using the best parameters, model will be trained/validated/tested using 5-fold CV and the whole process was repeated for 5 times. Results were then averaged and reported
3. Ensure that data folder, from_scratch_trained_models folder and this notebook are in the same working directory
4. run_training to repeat the training process, please create a new folder to retrain, save and load saved model
5. Else, comment the run_training function and do testing to get the results which were obtained in the paper. 

In [2]:
def run_training(train_loader, valid_loader, params, trained_model_path):
    model = GraphTrans(num_features=NUM_FEATURES, num_targets=NUM_TARGET, num_layers=params['num_layers'], hidden_size=params['hidden_size'], n_heads=params['n_heads'], dropout=params['dropout'], edge_dim=EDGE_DIM)
    model.to(DEVICE)
    optimizer=torch.optim.Adam(model.parameters(),lr = params['learning_rate'])
    eng = EngineHOB(model, optimizer, device=DEVICE)

    best_loss = np.inf
    early_stopping_iter = PATIENCE
    early_stopping_counter = 0 

    for epoch in range(EPOCHS):
        train_loss = eng.train(train_loader)
        valid_loss = eng.validate(valid_loader)
        print(f'Epoch: {epoch+1}/{EPOCHS}, train loss : {train_loss}, validation loss : {valid_loss}')
        if valid_loss < best_loss:
            best_loss = valid_loss 
            early_stopping_counter=0 #reset counter
            print('Saving model...')
            torch.save(model.state_dict(), trained_model_path)
        else:
            early_stopping_counter +=1

        if early_stopping_counter > early_stopping_iter:
            print('Early stopping...')
            break
        print(f'Early stop counter: {early_stopping_counter}')
    
    return best_loss

def run_testing(test_loader, params,  trained_model_path):
    model = GraphTrans(num_features=NUM_FEATURES, num_targets=NUM_TARGET, num_layers=params['num_layers'], hidden_size=params['hidden_size'], n_heads=params['n_heads'], dropout=params['dropout'], edge_dim=EDGE_DIM)
    model.load_state_dict(torch.load(trained_model_path))
    model.to(DEVICE)
    optimizer=torch.optim.Adam(model.parameters(),lr = params['learning_rate'])
    eng = EngineHOB(model, optimizer, device=DEVICE)

    print('Begin testing...')
    bce, acc, f1, roc_auc= eng.test(test_loader)
    print('Test completed!')
    print(f'bce:{bce}, acc:{acc}, f1: {f1}, roc_auc: {roc_auc}')
    return bce, acc, f1, roc_auc

In [4]:
params = params_gt
n_repetitions = 5
train_data_root_path = './data/graph_data/data_oral_avail_train/'
train_data_raw_filename = 'data_oral_avail_train_50.csv'
test_data_root_path = './data/graph_data/data_oral_avail_test/'
test_data_raw_filename = 'data_oral_avail_test_1_50.csv'
path_to_save_trained_model = './from_scratch_trained_models/GT/'

bce_list = []
acc_list = []
f1_list = []
roc_auc_list = []

#load dataset 
dataset_for_cv = LoadHOBDataset(root=train_data_root_path, raw_filename=train_data_raw_filename)
test_dataset = LoadHOBDataset(root=test_data_root_path, raw_filename=test_data_raw_filename)
kf = KFold(n_splits=N_SPLITS)

for repeat in range(n_repetitions):
    for fold_no, (train_idx, valid_idx) in enumerate(kf.split(dataset_for_cv)):
        seed_everything(SEED_NO)
        train_dataset= []
        valid_dataset = []
        for t_idx in train_idx:
            train_dataset.append(torch.load(f'./data/graph_data/data_oral_avail_train/processed/molecule_{t_idx}.pt'))
        for v_idx in valid_idx:
            valid_dataset.append(torch.load(f'./data/graph_data/data_oral_avail_train/processed/molecule_{v_idx}.pt'))

        train_loader = DataLoader(train_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True)
        valid_loader = DataLoader(valid_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False)
        test_loader = DataLoader(test_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False)

        run_training(train_loader, valid_loader, params, 
                            os.path.join(path_to_save_trained_model, f'gt_repeat_{repeat}_fold_{fold_no}.pt'))
        bce, acc, f1, roc_auc = run_testing(test_loader, params, os.path.join(path_to_save_trained_model, f'gt_repeat_{repeat}_fold_{fold_no}.pt'))
        bce_list.append(bce)
        acc_list.append(acc)
        f1_list.append(f1)
        roc_auc_list.append(roc_auc)

bce_arr = np.array(bce_list)
mean_bce = np.mean(bce_arr)
sd_bce = np.std(bce_arr)
print(f'bce:{mean_bce:.3f}±{sd_bce:.3f}')

acc_arr = np.array(acc_list)
acc_mean= np.mean(acc_arr)
acc_sd = np.std(acc_arr)
print(f'acc:{acc_mean:.3f}±{acc_sd:.3f}')

f1_arr = np.array(f1_list)
f1_mean= np.mean(f1_arr)
f1_sd = np.std(f1_arr)
print(f'f1: {f1_mean:.3f}±{f1_sd:.3f}')

roc_auc_arr = np.array(roc_auc_list)
roc_auc_mean= np.mean(roc_auc_arr)
roc_auc_sd = np.std(roc_auc_arr)
print(f'roc_auc: {roc_auc_mean:.3f}±{roc_auc_sd:.3f}')

Epoch: 1/300, train loss : 1.0818932354450226, validation loss : 0.9042943120002747
Saving model...
Early stop counter: 0
Epoch: 2/300, train loss : 0.8742576390504837, validation loss : 0.6444023251533508
Saving model...
Early stop counter: 0
Epoch: 3/300, train loss : 0.706589013338089, validation loss : 0.7645893692970276
Early stop counter: 1
Epoch: 4/300, train loss : 0.7244753837585449, validation loss : 0.697129487991333
Early stop counter: 2
Epoch: 5/300, train loss : 0.750020831823349, validation loss : 0.7543023228645325
Early stop counter: 3
Epoch: 6/300, train loss : 0.6954939365386963, validation loss : 0.7170941829681396
Early stop counter: 4
Epoch: 7/300, train loss : 0.6909188777208328, validation loss : 0.67427659034729
Early stop counter: 5
Epoch: 8/300, train loss : 0.6818986088037491, validation loss : 0.6691092252731323
Early stop counter: 6
Epoch: 9/300, train loss : 0.6825099289417267, validation loss : 0.6500078439712524
Early stop counter: 7
Epoch: 10/300, trai